This notebook generates the paragraph about the microCT-scanning from logfiles of the scans.

In [344]:
import platform
import os
import pandas
import glob

In [341]:
# Copied from https://codereview.stackexchange.com/a/229889
import traitlets
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog

class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        with out:
            try:
                # Create Tk root
                root = Tk()
                # Hide the main window
                root.withdraw()
                # Raise the root to the top of all windows.
                root.call('wm', 'attributes', '.', '-topmost', True)
                # List of selected fileswill be set to b.value
                b.files = filedialog.askopenfilename(multiple=True)

                b.description = "Files Selected"
                b.icon = "check-square-o"
                b.style.button_color = "lightgreen"
            except:
                pass
out = widgets.Output()
raw = SelectFilesButton()
widgets.VBox([raw, out])

We can also select a folder and go through *each* subfolder there...

In [622]:
# Different locations if running either on Linux or Windows
if 'Linux' in platform.system():
    BasePath = os.path.join(os.sep, 'home', 'habi', 'research-storage-uct', 'Archiv_Tape')
else:
    BasePath = os.path.join('R:\\')
Root = os.path.join(BasePath, 'Rabbit-Grenoble')
print('We are loading all the data from %s' % Root)

We are loading all the data from /home/habi/research-storage-uct/Archiv_Tape/Rabbit-Grenoble


In [711]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [712]:
# Look only for folders: https://stackoverflow.com/a/38216530
Data['Folder'] = sorted(glob.glob(os.path.join(Root, '**', 'Ra*' + os.path.sep)))
Data['Scan'] = [os.path.basename(os.path.split(f)[0]) for f in Data['Folder']]

In [713]:
# Grab a bunch of log files from a folder
# We could do it in a list comprehension, but then it fails if we're still scanning a tooth
# Data['LogFile'] = [sorted(glob.glob(os.path.join(f, '*.log')))[0] for f in Data['Folder']]
for c, row in Data.iterrows():
    try:
        Data.at[c,'LogFile'] = sorted(glob.glob(os.path.join(row['Folder'], 'proj', '*.log')))[0]
    except IndexError:
        print('No logfile found in %s, removing the folder temporarily' % row.Folder)
        Data.at[c,'LogFile'] = 'scanning'
Data = Data[Data['LogFile'] != 'scanning']
Data.reset_index(drop=True, inplace=True)
print('We have %s tooth folders to work with' % (len(Data)))

No logfile found in /home/habi/research-storage-uct/Archiv_Tape/Rabbit-Grenoble/Rabbit-2-Grenoble2015/Rabbit-2-Overview-Grenoble2015-13-3um/, removing the folder temporarily
We have 16 tooth folders to work with


In [614]:
# Choose the file selected above
log = raw.files[0]

In [615]:
# Choose the file selected above
log = Data.LogFile[0]

In [616]:
print('We are looking at the values in %s' % log)

We are looking at the values in /home/habi/research-storage-uct/Archiv_Tape/Rabbit-Grenoble/Rabbit-2-Grenoble2015/Rabbit-2-Overview-Grenoble2015-26-5um/proj/Rabbit-2-Overview-Grenoble2015-26-5um_.log


In [617]:
# Load log file manually (probably needed on Windows)
# log = 'f:\Verdiana Lung\Experiment01\Mouse04\proj\Mouse04.log'

In [618]:
log = '/home/habi/research-storage-uct/Archiv_Tape/Rabbit-Grenoble/Rabbit-2-Grenoble2015/Rabbit-2-Overview-Grenoble2015-26-5um/proj/Rabbit-2-Overview-Grenoble2015-26-5um_.log'

In [619]:
log = '/home/habi/1272/ZMK/ToothBattallion/99/Tooth099.log'

In [626]:
def fulllog(logfile):
    with open(logfile, 'r') as f:
        for line in f:
            print(line.strip())
    return()

In [627]:
# fulllog(log)

In [628]:
import re
def scanner(logfile, verbose=False):
    hardwareversion = []
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scanner' in line:
                if verbose:
                    print(line)
                # Sometimes it's SkyScan, sometimes Skyscan, so we have to regex it :)
                machine = re.split('Sky.can', line)[1].strip()
            if 'Hardware' in line:
                if verbose:
                    print(line)
                hardwareversion = line.split('=')[1].strip()
    if hardwareversion:
        return('SkyScan %s (Version %s)' % (machine, hardwareversion))
    else:
        return('SkyScan ' + machine)    

In [629]:
# scanner(log, verbose=True)

In [630]:
def scansoftware(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Software' in line:
                if verbose:
                    print(line)
                version = line.split('=')[1].strip()
    return(version)

In [631]:
# fulllog(log)

In [632]:
# scansoftware(log, verbose=True)

In [633]:
def source(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Source T' in line:
                if verbose:
                    print(line)
                # Sometimes it's SkyScan, sometimes Skyscan, so we have to regex it :)
                source = line.split('=')[1].strip()
    return(source)

In [634]:
# source(log, verbose=True)

In [635]:
def camera(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Camera T' in line or 'Camera=' in line:
                if verbose:
                    print(line)
                cam = line.split('=')[1].strip().strip(' camera')
    return(cam)

In [636]:
# camera(log, verbose=True)

In [637]:
def voltage(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Voltage' in line:
                if verbose:
                    print(line)
                V = float(line.split('=')[1])
    return(V)

In [638]:
def current(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Source Current' in line:
                if verbose:
                    print(line)
                A = float(line.split('=')[1])
    return(A)

In [639]:
# current(log, verbose=True)

In [640]:
def whichfilter(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Filter=' in line:
                if verbose:
                    print(line)
                fltr = line.split('=')[1].strip().replace('  ', ' ')
                if fltr=='No Filter':
                    fltr=False
    return(fltr)

In [641]:
# whichfilter(log, verbose=True)

In [642]:
def numproj(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'f Files' in line:
                if verbose:
                    print(line)
                numproj = int(line.split('=')[1])
    return(numproj)

In [643]:
# numproj(log, verbose=True)

In [644]:
def stacks(logfile, verbose=False):
    with open(logfile, 'r') as f:
        # If only one stack, then there's nothing in the log file
        numstacks = 0
        for line in f:
            if 'b-scan' in line:
                if verbose:
                    print(line)
                # The 'Sub-scan scan length' is listed in the log file
                # We simply select the last one, and add 1, since Bruker also starts to count at zero
                numstacks = int(line.split('[')[1].split(']')[0])
    return(numstacks + 1)

In [645]:
# stacks(log, verbose=True)

In [646]:
def camerasize(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Columns' in line:
                if verbose:
                    print(line)
                columns = int(line.split('=')[1])
            if 'Rows' in line:
                if verbose:
                    print(line)
                rows = int(line.split('=')[1])
    return(columns, rows)

In [647]:
def rotationstep(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Rotation Step' in line:
                if verbose:
                    print(line)
                rotstep = float(line.split('=')[1])
    return(rotstep)

In [648]:
def exposure(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Exposure' in line:
                if verbose:
                    print(line)
                exp = int(line.split('=')[1])
    return(exp)

In [649]:
def averaging(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Avera' in line:
                if verbose:
                    print(line)
                details = line.split('=')[1]
                if 'ON' in details:
                    # https://stackoverflow.com/a/4894156/323100
                    avg = int(details[details.find("(")+1:details.find(")")])
                else:
                    avg=False
    return(avg)

In [650]:
import datetime
def duration(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scan duration' in line:
                if verbose:
                    print(line)
                duration = line.split('=')[1].strip()
    # Sometimes it's '00:24:26', sometimes '0h:52m:53s'
    if 'h' in duration:
        return(datetime.datetime.strptime(duration, '%Hh:%Mm:%Ss'))
    else:
        return(datetime.datetime.strptime(duration, '%H:%M:%S'))

In [651]:
# print(duration(log, verbose=True).hour, 'hours')
# print(duration(log).minute, 'minutes')
# print(duration(log).second, 'seconds')

In [652]:
def pixelsize(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Image Pixel' in line and 'Scaled' not in line:
                if verbose:
                    print(line)
                pixelsize = float(line.split('=')[1])
    return(pixelsize)

In [653]:
def version(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Reconstruction Program' in line:
                if verbose:
                    print(line)
                Program = line.split('=')[1].strip()
            if 'Program Version' in line:
                if verbose:
                    print(line)
                Version = line.split('sion:')[1].strip()
    return(Program, Version)

In [748]:
def ringremoval(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Ring' in line:
                if verbose:
                    print(line)
                ring = int(line.split('=')[1].strip())
    return(ring)

In [750]:
# ringremoval(log, verbose=True)

In [742]:
def beamhardening(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'ardeni' in line:
                if verbose:
                    print(line)
                bh = int(line.split('=')[1].strip())
    return(bh)

In [751]:
# beamhardening(log, verbose=True)

----

My microct blurb from http://simp.ly/publish/NBhZhH

In [655]:
print('Based on %s' % log)

Based on /home/habi/1272/ZMK/ToothBattallion/99/Tooth099.log


In [656]:
print('After $PREPARATION, the $NUMSAMPLES samples were imaged on a Bruker',
      scanner(log),
      'high-resolution microtomography machine (Control software version',
      scansoftware(log) +
      ', Bruker microCT, Kontich, Belgium).')

After $PREPARATION, the $NUMSAMPLES samples were imaged on a Bruker SkyScan 1272 high-resolution microtomography machine (Control software version 1.1.19, Bruker microCT, Kontich, Belgium).


In [657]:
print('The machine is equipped with a',
      source(log, verbose=False),
      'X-ray source and a',
      camera(log) +
      ' camera.')

The machine is equipped with a HAMAMATSU_L11871_20 X-ray source and a XIMEA xiRAY16 camera.


In [658]:
print('The X-ray source was set to a tube voltage of', 
      voltage(log),
      'kV and a tube current of',
      current(log),
      'µA, the x-ray spectrum was', end=' ')
if whichfilter(log):
    print('filtered by', whichfilter(log), end=' ')
else:
    print('not filtered', end=' ')
print('prior to incidence onto the sample.')

The X-ray source was set to a tube voltage of 80.0 kV and a tube current of 125.0 µA, the x-ray spectrum was filtered by Al 1mm prior to incidence onto the sample.


In [659]:
print('For each sample, we recorded a set of', end=' ')
if stacks(log) > 1:
    print(stacks(log), 'stacked scans overlapping the sample height, each stack with', end=' ')
print(numproj(log),
      'projections of',
      camerasize(log)[0],
      'x',
      camerasize(log)[1],
      'pixels at every',
      str(rotationstep(log)) + '° over a 180° sample rotation.')

For each sample, we recorded a set of 5 stacked scans overlapping the sample height, each stack with 482 projections of 1632 x 1092 pixels at every 0.4° over a 180° sample rotation.


In [660]:
print('Every single projection was exposed for',
      exposure(log),
      'ms,',
      averaging(log),
      'projections were averaged to one to greatly reduce image noise.')

Every single projection was exposed for 950 ms, 3 projections were averaged to one to greatly reduce image noise.


In [661]:
print('This resulted in scan times of approximately ', end='')
if duration(log).hour:
    print(duration(log).hour, 'hours', end=' ')
print(round(duration(log).minute/5)*5,
      'minutes per stack, and an isometric voxel size of',
      round(pixelsize(log),1),
      'µm in the final data sets.')

This resulted in scan times of approximately 40 minutes per stack, and an isometric voxel size of 10.0 µm in the final data sets.


In [747]:
print('The projection images were then subsequently reconstructed into a 3D stack',
      'of images with',
      version(log)[0],
      '(Version',
      version(log)[1],
      ', Bruker microCT, Kontich Belgium) with a ring artifact correction of',
      ringremoval(log), end='')
if beamhardening(log) > 0:
    print(' and a beam hardening correction of',
          beamhardening(log),
          '%.')
else:
    print('.')

The projection images were then subsequently reconstructed into a 3D stack of images with NRecon (Version 1.7.4.6 , Bruker microCT, Kontich Belgium) with a ring artifact correction of 14.


In [663]:
# fulllog(log)

----

If we have several, then generate a dataframe...

In [714]:
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['Software'] = [scansoftware(log) for log in Data['LogFile']]

In [715]:
Data['Voxelsize'] = [pixelsize(log) for log in Data['LogFile']]

In [716]:
Data['Source'] = [source(log) for log in Data['LogFile']]
Data['Camera'] = [camera(log) for log in Data['LogFile']]

In [717]:
Data['Voltage'] = [voltage(log) for log in Data['LogFile']]
Data['Current'] = [current(log) for log in Data['LogFile']]
Data['Filter'] = [whichfilter(log) for log in Data['LogFile']]

In [718]:
Data['Stacks'] = [stacks(log) for log in Data['LogFile']]
Data['NumProj'] = [numproj(log) for log in Data['LogFile']]
Data['CamSize'] = [camerasize(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]

In [752]:
Data['RingRemoval'] = [ringremoval(log) for log in Data['LogFile']]
Data['Beamhardening'] = [beamhardening(log) for log in Data['LogFile']]

In [719]:
Data['Exposure'] = [exposure(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]

In [720]:
Data['Duration'] = [duration(log) for log in Data['LogFile']]

In [721]:
Data['Version'] = [version(log) for log in Data['LogFile']]

In [722]:
Data.to_excel('Rabbits.xls')

In [753]:
Data.head()

,Folder,Scan,LogFile,Scanner,Software,Voxelsize,Source,Camera,Voltage,Current,...,Stacks,NumProj,CamSize,RotationStep,Exposure,Averaging,Duration,Version,RingRemoval,Beamhardening
0,/home/habi/research-storage-uct/Archiv_Tape/Ra...,Rabbit-1-Overview--Mandibula-Grenoble2015-26-6um,/home/habi/research-storage-uct/Archiv_Tape/Ra...,SkyScan 1172 (Version F),Version 1. 5 (build 20),26.54,Hamamatsu 100/250,Hamamatsu C9300 11Mp,100.0,100.0,...,4,583,"(1984, 668)",0.35,300,2,1900-01-01 00:24:22,"(NRecon, 1.7.4.2)",7,40
1,/home/habi/research-storage-uct/Archiv_Tape/Ra...,Rabbit-1-Overview-Maxilaris-Grenoble2015-13-3um,/home/habi/research-storage-uct/Archiv_Tape/Ra...,SkyScan 1172 (Version F),Version 1. 5 (build 20),13.27,Hamamatsu 100/250,Hamamatsu C9300 11Mp,100.0,100.0,...,4,1018,"(3968, 1336)",0.20,1250,3,1900-01-01 02:50:17,"(NRecon, 1.7.4.2)",7,40
2,/home/habi/research-storage-uct/Archiv_Tape/Ra...,Rabbit-1-Overview-Maxilaris-Grenoble2015-26-6um,/home/habi/research-storage-uct/Archiv_Tape/Ra...,SkyScan 1172 (Version F),Version 1. 5 (build 20),26.54,Hamamatsu 100/250,Hamamatsu C9300 11Mp,100.0,100.0,...,4,583,"(1984, 668)",0.35,300,2,1900-01-01 00:24:26,"(NRecon, 1.7.4.2)",7,40
3,/home/habi/research-storage-uct/Archiv_Tape/Ra...,Rabbit-2-Grenoble2015-26-5um,/home/habi/research-storage-uct/Archiv_Tape/Ra...,SkyScan 1172 (Version F),Version 1. 5 (build 20),26.54,Hamamatsu 100/250,Hamamatsu C9300 11Mp,100.0,100.0,...,1,583,"(1984, 668)",0.35,300,2,1900-01-01 00:24:51,"(NRecon, 1.6.9.9)",9,40
4,/home/habi/research-storage-uct/Archiv_Tape/Ra...,Rabbit-2-Overview-Grenoble2015-26-5um,/home/habi/research-storage-uct/Archiv_Tape/Ra...,SkyScan 1172 (Version F),Version 1. 5 (build 20),26.54,Hamamatsu 100/250,Hamamatsu C9300 11Mp,100.0,100.0,...,3,583,"(1984, 668)",0.35,300,2,1900-01-01 00:24:23,"(NRecon, 1.7.4.2)",7,40


In [728]:
Data.groupby(by=['Voxelsize']).describe()

Voltage                                                Current  \
            count   mean  std    min    25%    50%    75%    max   count   
Voxelsize                                                                  
13.27         4.0  100.0  0.0  100.0  100.0  100.0  100.0  100.0     4.0   
26.54        12.0  100.0  0.0  100.0  100.0  100.0  100.0  100.0    12.0   

                  ... Exposure         Averaging                           \
            mean  ...      75%     max     count mean  std  min  25%  50%   
Voxelsize         ...                                                       
13.27      100.0  ...   1250.0  1250.0       4.0  3.0  0.0  3.0  3.0  3.0   
26.54      100.0  ...    300.0   300.0      12.0  2.0  0.0  2.0  2.0  2.0   

                     
           75%  max  
Voxelsize            
13.27      3.0  3.0  
26.54      2.0  2.0  

[2 rows x 56 columns]

In [727]:
Data.Voxelsize[5]

26.54

In [729]:
log

'/home/habi/1272/ZMK/ToothBattallion/99/Tooth099.log'

In [730]:
fulllog(log)

[System]
Scanner=SkyScan1272
Instrument S/N=15G09089-B
Software Version=1.1.19
Home Directory=C:\SkyScan1272\SkyScan1272
Source Type=HAMAMATSU_L11871_20
Camera Type=XIMEA xiRAY16
Camera Pixel Size (um)=7.4
Camera X/Y Ratio=1.0023
[User]
User Name=haberthu
Computer Name=ANAMIC02
[Acquisition]
Data Directory=D:\Results\ZMK\ToothBattallion\99
Filename Prefix=Tooth099~00
Filename Index Length=8
Number Of Files=  482
Number Of Rows= 1092
Number Of Columns= 1632
Partial Width=OFF
Image crop origin X=0
Image crop origin Y=0
Camera binning=3x3
Image Rotation=0.13600
Optical Axis (line)=  580
Camera to Source (mm)=174.07267
Object to Source (mm)=78.41100
Source Voltage (kV)=  80
Source Current (uA)= 125
Image Pixel Size (um)=9.999986
Scaled Image Pixel Size (um)=9.999986
Image Format=TIFF
Depth (bits)=16
Reference Intensity=57000
Camera position=close
Exposure (ms)=950
Rotation Step (deg)=0.400
Use 360 Rotation=NO
Scanning position=36.000 mm
Frame Averaging=ON (3)
Random Movement=OFF (30)
Flat 

()